In [1]:
import pandas as pd
import numpy as np
import requests
import os
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Set Fantasy Points

In [2]:
qb_fantasy_points = {
            'touchdown': 6,
            'pass_yds': 25,
            'interception': -2,
            'rush_yds': 10,
            'rush_td': 6,
        }

flex_fantasy_points = {
            'rush_touchdown': 6,
            'rush_yards': 10,
            'receptions': 1, # ppr
            'rec_yards': 10,
            'rec_touchdown': 6,
        }

kicking_fantasy_points = {
            'field_goals_made': 3,
            'field_goals_missed': -1,
            'extra_points_made': 1,
            'extra_points_missed': -1,
        }

# Scrape Data
This scrapes all data, for all positions in `starting_week` through `current_week`

In [3]:
# Set the weeks in which you would like to pull data
starting_week = 19
current_week = 23

In [4]:
all_positions = ['QB', 'WR', 'RB', 'TE', 'K']
weeks = [week for week in range(starting_week, current_week + 1)]

file_list = []

for position in all_positions:
    for week in weeks:
        url = f'https://www.cbssports.com/nfl/stats/leaders/live/{position}/{week}/'
        res = requests.get(url)
        output = open(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls', 'wb')
        output.write(res.content)
        file_list.append(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        output.close()
        data = pd.read_html(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        df = data[0]

In [5]:
position_list = all_positions
dfs = []
for file in file_list:
    for i in range(len(position_list)):
        if position_list[i] in file:
            data = pd.read_html(file)
            df2 = pd.DataFrame(data[0])
            if position_list[i] != 'K':
                df2 = df2.droplevel(0, axis=1)
                df2['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
                df2['Pos'] = [i.split()[-1] for i in df2['Player  Player on team']]
                df2.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
                df2['Player'] = df2['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
            else:
                df2['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
                df2['Pos'] = [i.split()[-1] for i in df2['Player  Player on team']]
                df2.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
                df2['Player'] = df2['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
            dfs.append(df2)
df = pd.concat(dfs, ignore_index=True)

In [6]:
def impute_special_char(df, char):
    df.replace(char, 0, inplace = True) # get rid of the dashes, impute with 0
    
impute_special_char(df, '—')

In [7]:
def change_col_types(df):
    numcols_to_change = df.columns
    numcols_to_change2 = []
    for col in numcols_to_change:
        try:
            df[col] = df[col].astype(int)
            print('success!')
        except:
            numcols_to_change2.append(col)
            print(f'need to clean column: {col}')
            
change_col_types(df)

need to clean column: Player
need to clean column: Game
success!
need to clean column: CMP  Pass Completions
need to clean column: ATT  Pass Attempts
need to clean column: YDS  Passing Yards
need to clean column: TD  Touchdown Passes
need to clean column: INT  Interceptions Thrown
need to clean column: ATT  Rushing Attempts
need to clean column: YDS  Rushing Yards
need to clean column: TD  Rushing Touchdowns
success!
need to clean column: Pos
need to clean column: REC  Receptions
need to clean column: YDS  Receiving Yards
need to clean column: TGT  Targets
need to clean column: TD  Receiving Touchdowns
need to clean column: FGM-A  Field Goals Made - Field Goals Attempted
need to clean column: LNG  Longest field goal in terms of yards by a kicker
need to clean column: XPM-A  Extra Points Made - Extra Points Attempted


In [8]:
df.fillna(0, inplace=True)

In [9]:
df.shape

(312, 20)

In [11]:
# df['FG_Made'] = df['FGM-A  Field Goals Made - Field Goals Attempted'].map(lambda x: x[0])
# df['FG_Attempted'] = df['FGM-A  Field Goals Made - Field Goals Attempted'].map(lambda x: x[-1])
# df['XPM'] = df['XPM-A  Extra Points Made - Extra Points Attempted'].map(lambda x: x[0])
# df['XPA'] = df['XPM-A  Extra Points Made - Extra Points Attempted'].map(lambda x: x[-1])

### Scrape football reference, to get player information

In [12]:
position = ['rushing', 'passing', 'receiving']

for element in position:
    url = f'https://www.pro-football-reference.com/years/2021/{element}.htm'
    resp = requests.get(url)
    
    with open(f'./nfl_data/2021_{element}_stats.xls', 'wb') as f:
        f.write(resp.content)

In [13]:
pi_wr = pd.read_html('./nfl_data/2021_receiving_stats.xls')
pi_wr = pd.DataFrame(pi_wr[0]) # Saves df var to dataframe

pi_qb = pd.read_html('./nfl_data/2021_passing_stats.xls')
pi_qb = pd.DataFrame(pi_qb[0]) # Saves df var to dataframe

pi_rb = pd.read_html('./nfl_data/2021_rushing_stats.xls')
pi_rb = pd.DataFrame(pi_rb[0]) # Saves df var to dataframe
pi_rb = pi_rb.droplevel(0, axis=1) # Removes first level column

In [14]:
def drop_rows(position):
    for header in position.columns:
        index_list = position.loc[(position[header] == header)].index
        position.drop(labels=index_list, axis=0, inplace = True)
        return
    
drop_rows(pi_rb)
drop_rows(pi_wr)
drop_rows(pi_qb)

In [15]:
pi_wr['Player'] = pi_wr['Player'].map(lambda x: x.rstrip('_+!*@#$?^'))
pi_qb['Player'] = pi_qb['Player'].map(lambda x: x.rstrip('_+!*@#$?^'))
pi_rb['Player'] = pi_rb['Player'].map(lambda x: x.rstrip('_+!*@#$?^'))

In [16]:
dfs = [pi_wr, pi_qb, pi_rb]
player_info = pd.concat([player_info.squeeze() for player_info in dfs], ignore_index=True)

In [17]:
player_info = player_info[['Player', 'Tm']].copy()

In [18]:
player_info.rename(columns = {'Tm':'Team_Name_Abbrev'}, inplace = True)

In [19]:
player_info = player_info.drop_duplicates()

In [20]:
player_info_dict = pd.Series(player_info.Team_Name_Abbrev.values,index=player_info.Player).to_dict()
# create dictionary with {player : team abbreviation}

In [21]:
Team_Abbreviations_Dict = {
    'Arizona Cardinals': 'ARI',
    'Atlanta Falcons': 'ATL',
    'Baltimore Ravens': 'BAL',
    'Buffalo Bills' : 'BUF',
    'Carolina Panthers': 'CAR',
    'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN',
    'Cleveland Browns': 'CLE',
    'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN',
    'Detroit Lions': 'DET',
    'Green Bay Packers': 'GNB',
    'Houston Texans': 'HOU',
    'Indianapolis Colts': 'IND',
    'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KAN',
    'Miami Dolphins': 'MIA',
    'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NWE',
    'New Orleans Saints': 'NO',
    'NY Giants': 'NYG',
    'NY Jets': 'NYJ',
    'Las Vegas Raiders': 'LVR',
    'Philadelphia Eagles': 'PHI',
    'Pittsburgh Steelers': 'PIT',
    'Los Angeles Chargers': 'LAC',
    'San Francisco 49ers': 'SFO',
    'Seattle Seahawks': 'SEA',
    'Los Angeles Rams': 'LAR',
    'Tampa Bay Buccaneers': 'TAM',
    'Tennessee Titans': 'TEN',
    'Washington Commanders': 'WAS'
}

In [22]:
def get_key(val):
    for key, value in Team_Abbreviations_Dict.items():
         if val == value:
            return key
 
    return f'key does not exist --> {val}'

In [23]:
player_info['Team_Name_Full'] = player_info['Team_Name_Abbrev'].map(get_key)

In [24]:
player_info['Team_Name_Mascot'] = player_info['Team_Name_Full'].map(lambda x: x.split()[-1])

In [25]:
df.shape

(312, 20)

### Merge `player_info` with `df`

In [26]:
df = pd.merge(df, player_info, on = 'Player', how = 'left')
df['Team1'] = df['Game'].map(lambda x: x.split()[0])
df['Team2'] = df['Game'].map(lambda x: x.split()[-2])
df.head()

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,Team_Name_Abbrev,Team_Name_Full,Team_Name_Mascot,Team1,Team2
0,Josh Allen,BUF 47 - NE 17,48,21,25,308,5,0,6,66,0,19,QB,0,0,0,0,0,0.0,0,BUF,Buffalo Bills,Bills,BUF,NE
1,Patrick Mahomes,KC 42 - PIT 21,46,30,39,404,5,1,3,29,0,19,QB,0,0,0,0,0,0.0,0,KAN,Kansas City Chiefs,Chiefs,KC,PIT
2,Matthew Stafford,LAR 34 - ARI 11,28,13,17,202,2,0,6,22,1,19,QB,0,0,0,0,0,0.0,0,LAR,Los Angeles Rams,Rams,LAR,ARI
3,Dak Prescott,SF 23 - DAL 17,22,23,43,254,1,1,4,27,1,19,QB,0,0,0,0,0,0.0,0,DAL,Dallas Cowboys,Cowboys,SF,DAL
4,Tom Brady,TB 31 - PHI 15,22,29,37,271,2,0,0,0,0,19,QB,0,0,0,0,0,0.0,0,TAM,Tampa Bay Buccaneers,Buccaneers,TB,PHI


In [28]:
df.shape

(312, 25)

In [53]:
df['Opp'] = np.where(df['Team_Name_Abbrev'] == df['Team1'], df['Team2'], df['Team1'])
df.head()

## Scrape weather

In [ ]:
weeks = [f'week-{week}' for week in range(1,17)]

file_list = []

for week in weeks:
    url = f'https://www.nflweather.com/en/week/2021/{week}/'
    res = requests.get(url)
    output = open(f'./nfl_data/weekly/weather/{week}_weather.xls', 'wb')
    output.write(res.content)
    file_list.append(f'./nfl_data/weekly/weather/{week}_weather.xls')
    output.close()
    data = pd.read_html(f'./nfl_data/weekly/weather/{week}_weather.xls')
    df_w = data[0]
    print(f'Scraping weather for {week}')

Scraping weather for week-1
Scraping weather for week-2
Scraping weather for week-3
Scraping weather for week-4
Scraping weather for week-5
Scraping weather for week-6
Scraping weather for week-7
Scraping weather for week-8
Scraping weather for week-9
Scraping weather for week-10
Scraping weather for week-11
Scraping weather for week-12
Scraping weather for week-13
Scraping weather for week-14
Scraping weather for week-15
Scraping weather for week-16


In [ ]:
dfs = []
for file in file_list:
    data = pd.read_html(file)
    df2 = pd.DataFrame(data[0])
    df2.drop(columns = ['Unnamed: 0', 'Game', 'Game.1', 'Game.2', 'Time (ET)', 'TV', 'Unnamed: 8', 'Unnamed: 12'], inplace = True) 
    df2['Week'] = re.search(r'(\d+)', file)[0]
    df2['Wind_Speed_MPH'] = df2['Wind'].map(lambda x:re.search(r'(\d+)',x)[0])
    df2['Wind_Direction'] = df2['Wind'].map(lambda x:re.search(r'[A-Z]+',x)[0])
    df2['Temp'] = df2['Forecast'].map(lambda x: re.search(r'(\d+)',x)[0] if x != 'DOME' else 0)
    df2['Weather_Desc'] = df2['Forecast'].map(lambda x: re.search(r'\s(.+)',x)[0].strip() if x != 'DOME' else 0)
    df2["Wind"] = np.where(df2["Forecast"] == "DOME", 0, df2["Wind"])
    df2["Wind_Speed_MPH"] = np.where(df2["Forecast"] == "DOME", 0, df2["Wind_Speed_MPH"])
    df2["Wind_Direction"] = np.where(df2["Forecast"] == "DOME", 0, df2["Wind_Direction"])
    dfs.append(df2)
weather_df = pd.concat(dfs, ignore_index=True)
weather_df.head()

,Away,Home,Forecast,Extended Forecast,Wind,Week,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc
0,Cowboys,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,1,6,WSW,80,Humid and Partly Cloudy
1,Eagles,Falcons,DOME,Clear. Clear throughout the day.,0,1,0,0,0,0
2,Steelers,Bills,72f Mostly Cloudy,Mostly Cloudy. Rain overnight.,12m WSW,1,12,WSW,72,Mostly Cloudy
3,Vikings,Bengals,82f Clear,Clear. Clear throughout the day.,11m SW,1,11,SW,82,Clear
4,49ers,Lions,DOME,Mostly Cloudy. Mostly cloudy throughout the day.,0,1,0,0,0,0


In [35]:
weather_df['Week'] = weather_df['Week'].astype(int)

## Merge `df` and `df_weather`

In [36]:
change_col_types(df)

need to clean column: Player
need to clean column: Game
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
need to clean column: Pos
success!
success!
success!
success!
need to clean column: FGM-A  Field Goals Made - Field Goals Attempted
success!
need to clean column: XPM-A  Extra Points Made - Extra Points Attempted
need to clean column: Team_Name_Abbrev
need to clean column: Team_Name_Full
need to clean column: Team_Name_Mascot
need to clean column: Team1
need to clean column: Team2


In [37]:
df.shape

(312, 25)

In [38]:
# df.loc[df.Player == 'Tom Brady']

# Change TB12 to Week 1 for upcoming merge testing
# df.at[4,'Week']=1 # Change a week to ensure the join is working properly

In [39]:
df = pd.merge(df, weather_df, left_on = ['Team_Name_Mascot', 'Week'], right_on = ['Home', 'Week'], how = 'left')

In [40]:
df.shape

(312, 34)

In [41]:
df['Away_Game'] = np.where(df['Away'] == df['Team_Name_Mascot'], 1, 0)
df['Home_Game'] = np.where(df['Home'] == df['Team_Name_Mascot'], 1, 0)

In [42]:
df.fillna(0, inplace=True)

In [43]:
change_col_types(df)

need to clean column: Player
need to clean column: Game
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
need to clean column: Pos
success!
success!
success!
success!
need to clean column: FGM-A  Field Goals Made - Field Goals Attempted
success!
need to clean column: XPM-A  Extra Points Made - Extra Points Attempted
need to clean column: Team_Name_Abbrev
need to clean column: Team_Name_Full
need to clean column: Team_Name_Mascot
need to clean column: Team1
need to clean column: Team2
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!


In [44]:
# Test case - changes record to Away = Buccaneers, re-runs the boolean column calc. If you run this, run the next cell to return things to normal
# df.at[0,'Away']='Buccaneers'
# df['Away_Game'] = np.where(df['Away'] == df['Team_Name_Mascot'], 1, 0)
# df

In [45]:
# df.at[0,'Away']='Cowboys'
# df['Away_Game'] = np.where(df['Away'] == df['Team_Name_Mascot'], 1, 0)
# df

In [46]:
url = 'https://www.pro-football-reference.com/years/2021/opp.htm'
res = requests.get(url)

with open('./nfl_data/weekly/defensive_ranks.xls', 'wb') as f:
    f.write(res.content)

In [47]:
df_def = pd.read_html('./nfl_data/weekly/defensive_ranks.xls')
df_def = pd.DataFrame(df_def[0]) 
df_def = df_def.droplevel(0, axis=1)
df_def = df_def.rename(columns = {'Tm':'Team', 'Rk': 'Rank'})
df_def = df_def.iloc[:, [0,1]].copy() # filter to only Rank and Team
df_def['Tm_Abr'] = df_def['Team'].map(lambda x: Team_Abbreviations_Dict.get(x)) # tack on column with corresponding team abbreviation
df_def.drop(index = [32,33,34], axis = 0, inplace = True) # drop the average and total rows
df_def['Current_Week'] = current_week
df_def['Team_Mascot'] = df_def['Team'].map(lambda x: x.split()[-1])
df_def.head()

,Rank,Team,Tm_Abr,Current_Week,Team_Mascot
0,1.0,Buffalo Bills,BUF,23,Bills
1,2.0,New England Patriots,NWE,23,Patriots
2,3.0,Denver Broncos,DEN,23,Broncos
3,4.0,New Orleans Saints,NO,23,Saints
4,5.0,Tampa Bay Buccaneers,TAM,23,Buccaneers


In [48]:
# df['Opp'] = np.where(df['Team_Name_Mascot'] == df['Home'], df['Away'], df['Home'])
# df['Opp_Def_Rank'] = df['Opp'].map(lambda x: df_def.loc[df_def.Team_Mascot == x]['Rank'].values[0] if df['Opp'].notnull() else 0)

In [49]:
df.shape

(312, 36)

In [50]:
df.head()

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,Team_Name_Abbrev,Team_Name_Full,Team_Name_Mascot,Team1,Team2,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc,Away_Game,Home_Game
0,Josh Allen,BUF 47 - NE 17,48,21,25,308,5,0,6,66,0,19,QB,0,0,0,0,0,0,0,BUF,Buffalo Bills,Bills,BUF,NE,0,0,0,0,0,0,0,0,0,0,0
1,Patrick Mahomes,KC 42 - PIT 21,46,30,39,404,5,1,3,29,0,19,QB,0,0,0,0,0,0,0,KAN,Kansas City Chiefs,Chiefs,KC,PIT,0,0,0,0,0,0,0,0,0,0,0
2,Matthew Stafford,LAR 34 - ARI 11,28,13,17,202,2,0,6,22,1,19,QB,0,0,0,0,0,0,0,LAR,Los Angeles Rams,Rams,LAR,ARI,0,0,0,0,0,0,0,0,0,0,0
3,Dak Prescott,SF 23 - DAL 17,22,23,43,254,1,1,4,27,1,19,QB,0,0,0,0,0,0,0,DAL,Dallas Cowboys,Cowboys,SF,DAL,0,0,0,0,0,0,0,0,0,0,0
4,Tom Brady,TB 31 - PHI 15,22,29,37,271,2,0,0,0,0,19,QB,0,0,0,0,0,0,0,TAM,Tampa Bay Buccaneers,Buccaneers,TB,PHI,0,0,0,0,0,0,0,0,0,0,0


# Calculate Custom Fantasy Points

In [51]:
df['Fantasy_Pts'] = df['TD  Touchdown Passes'].multiply(qb_fantasy_points['touchdown']) + df['YDS  Passing Yards'].divide(qb_fantasy_points['pass_yds']) + df['INT  Interceptions Thrown'].multiply(qb_fantasy_points['interception']) + df['YDS  Rushing Yards'].divide(qb_fantasy_points['rush_yds']) + df['TD  Rushing Touchdowns'].multiply(qb_fantasy_points['rush_td'])


In [ ]:
df['Fantasy_Pts'] = df['TD  Rushing Touchdowns'].multiply(flex_fantasy_points['rush_touchdown']) + df['YDS  Rushing Yards'].divide(flex_fantasy_points['rush_yards']) + df['REC  Receptions'].add(flex_fantasy_points['receptions']) + df['YDS  Receiving Yards'].divide(flex_fantasy_points['rec_yards']) + df['TD  Receiving Touchdowns'].multiply(flex_fantasy_points['rec_touchdown'])


In [ ]:
df['Fantasy_Pts'] = df['TD  Rushing Touchdowns'].multiply(flex_fantasy_points['rush_touchdown']) + df['YDS  Rushing Yards'].divide(flex_fantasy_points['rush_yards']) + df['REC  Receptions'].add(flex_fantasy_points['receptions']) + df['YDS  Receiving Yards'].divide(flex_fantasy_points['rec_yards']) + df['TD  Receiving Touchdowns'].multiply(flex_fantasy_points['rec_touchdown'])


In [ ]:
df['Fantasy_Pts'] = df['TD  Rushing Touchdowns'].multiply(flex_fantasy_points['rush_touchdown']) + df['YDS  Rushing Yards'].divide(flex_fantasy_points['rush_yards']) + df['REC  Receptions'].add(flex_fantasy_points['receptions']) + df['YDS  Receiving Yards'].divide(flex_fantasy_points['rec_yards']) + df['TD  Receiving Touchdowns'].multiply(flex_fantasy_points['rec_touchdown'])


In [52]:
df.head(20)

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,Team_Name_Abbrev,Team_Name_Full,Team_Name_Mascot,Team1,Team2,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc,Away_Game,Home_Game,Fantasy_Pts
0,Josh Allen,BUF 47 - NE 17,48,21,25,308,5,0,6,66,0,19,QB,0,0,0,0,0,0,0,BUF,Buffalo Bills,Bills,BUF,NE,0,0,0,0,0,0,0,0,0,0,0,48.92
1,Patrick Mahomes,KC 42 - PIT 21,46,30,39,404,5,1,3,29,0,19,QB,0,0,0,0,0,0,0,KAN,Kansas City Chiefs,Chiefs,KC,PIT,0,0,0,0,0,0,0,0,0,0,0,47.06
2,Matthew Stafford,LAR 34 - ARI 11,28,13,17,202,2,0,6,22,1,19,QB,0,0,0,0,0,0,0,LAR,Los Angeles Rams,Rams,LAR,ARI,0,0,0,0,0,0,0,0,0,0,0,28.28
3,Dak Prescott,SF 23 - DAL 17,22,23,43,254,1,1,4,27,1,19,QB,0,0,0,0,0,0,0,DAL,Dallas Cowboys,Cowboys,SF,DAL,0,0,0,0,0,0,0,0,0,0,0,22.86
4,Tom Brady,TB 31 - PHI 15,22,29,37,271,2,0,0,0,0,19,QB,0,0,0,0,0,0,0,TAM,Tampa Bay Buccaneers,Buccaneers,TB,PHI,0,0,0,0,0,0,0,0,0,0,0,22.84
5,Joe Burrow,CIN 26 - LV 19,21,24,34,244,2,0,2,-2,0,19,QB,0,0,0,0,0,0,0,CIN,Cincinnati Bengals,Bengals,CIN,LV,0,0,0,0,0,0,0,0,0,0,0,21.56
6,Ben Roethlisberger,KC 42 - PIT 21,20,29,44,215,2,0,2,-1,0,19,QB,0,0,0,0,0,0,0,PIT,Pittsburgh Steelers,Steelers,KC,PIT,0,0,0,0,0,0,0,0,0,0,0,20.50
7,Mac Jones,BUF 47 - NE 17,18,24,38,232,2,2,2,18,0,19,QB,0,0,0,0,0,0,0,NWE,New England Patriots,Patriots,BUF,NE,0,0,0,0,0,0,0,0,0,0,0,19.08
8,Jalen Hurts,TB 31 - PHI 15,17,23,43,258,1,2,8,39,0,19,QB,0,0,0,0,0,0,0,PHI,Philadelphia Eagles,Eagles,TB,PHI,0,0,0,0,0,0,0,0,0,0,0,16.22
9,Derek Carr,CIN 26 - LV 19,16,29,54,310,1,1,1,20,0,19,QB,0,0,0,0,0,0,0,LVR,Las Vegas Raiders,Raiders,CIN,LV,0,0,0,0,0,0,0,0,0,0,0,18.40
